In [11]:
from datetime import datetime

import torch
from loguru import logger

from tentamen.data import datasets
from tentamen.model import Accuracy
from tentamen.settings import presets
from tentamen.train import trainloop

if __name__ == "__main__":
    logger.add(presets.logdir / "01.log")

    trainstreamer, teststreamer = datasets.get_arabic(presets)

    from tentamen.model import GRUmodel, Linear, GRUClassifier
    from tentamen.settings import LinearConfig, GruConfig

    configs = [
        LinearConfig(
            input=13, output=20, tunedir=presets.logdir, h1=100, h2=10, dropout=0.5
        ),
        GruConfig(
            input=13, output=10,tunedir=presets.logdir, num_layers=2, hidden_size=32, dropout=0.2
        )
    ]

rnn = torch.nn.RNN(
    input_size=1,
    hidden_size=10,
    num_layers=3,
    batch_first=True)
out, hidden = rnn(x)
out.shape, hidden.shape


2023-01-23 11:44:10.847 | INFO     | tentamen.data.data_tools:get_file:61 - File /home/azureuser/code/ML22-tentamen/data/raw/ArabicTrain.txt already exists, skip download
2023-01-23 11:44:10.850 | INFO     | tentamen.data.data_tools:get_file:61 - File /home/azureuser/code/ML22-tentamen/data/raw/ArabicTest.txt already exists, skip download
2023-01-23 11:44:10.851 | INFO     | tentamen.data.datasets:get_arabic:34 - Loading data from /home/azureuser/code/ML22-tentamen/data/raw/ArabicTrain.txt and /home/azureuser/code/ML22-tentamen/data/raw/ArabicTest.txt
2023-01-23 11:44:11.949 | INFO     | tentamen.data.datasets:get_arabic:50 - Returning trainstreamer, teststreamer


NameError: name 'x' is not defined

In [7]:
import torch.nn as nn
from typing import Callable, Dict, Protocol

Tensor = torch.Tensor

class GRUmodel(nn.Module):
    def __init__(self, config: Dict) -> None:
        super(GRUmodel, self).__init__()
        self.rnn = nn.GRU(
            input_size=config["input"],
            hidden_size=config["hidden_size"],
            dropout=config["dropout"],
            batch_first=True,
            num_layers=config["num_layers"],
        )
        self.linear1 = nn.Linear(config["hidden_size"], config["output"])
        self.linear2 = nn.Linear(config["output"], config["num_classes"])
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x: Tensor) -> Tensor:
        print("the shape is",x.shape)
        x, _ = self.rnn(x)
        last_step = x[:, -1, :].squeeze() 
        yhat = self.linear1(last_step)
        yhat = self.relu(yhat)
        yhat = self.linear2(yhat)
        yhat = self.softmax(yhat)
        return yhat

In [8]:
config =  GruConfig(input=13, output=10,tunedir=presets.logdir, num_layers=2, hidden_size=32, dropout=0.2)

config_GRU = {
        "input": 13,
        "hidden_size": 64,
        "dropout": 0,
        "num_layers": 1,
        "output": 32,
        "num_classes": 10
    }

model_gru = GRUmodel(config_GRU)
  #  for config in configs:
      #  model = Linear(config.dict())  # type: ignore
       # model_gru = GRUmodel(config.dict())

trainedmodel = trainloop(
            epochs=10,
            model=model_gru,  # type: ignore
            optimizer=torch.optim.Adam,
            learning_rate=1e-3,
            loss_fn=torch.nn.CrossEntropyLoss(),
            metrics=[Accuracy()],
            train_dataloader=trainstreamer.stream(),
            test_dataloader=teststreamer.stream(),
            log_dir=presets.logdir,
            train_steps=len(trainstreamer),
            eval_steps=len(teststreamer),
        )

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
path = presets.modeldir / (timestamp + presets.modelname)
logger.info(f"save model to {path}")
torch.save(trainedmodel, path)


2023-01-23 11:15:11.138 | INFO     | tentamen.train:dir_add_timestamp:159 - Logging to /home/azureuser/code/ML22-tentamen/logs/20230123-1115
  0%|          | 0/10 [00:00<?, ?it/s]


the shape is torch.Size([128, 63, 13])


IndexError: Target 11 is out of bounds.